# Dimer Builder Web App
    
This notebook helps you:
- Load one or two monomers (XYZ).
- Inspect monomer orientation (with long XYZ axes).
- Generate dimer grids via translations (x, y, z) and rotations (Rx, Ry, Rz).
- Preview/export resulting dimers (XYZ).
- **Optionally** run xTB single-point energies over all dimers.
    
> Rotation order used below is **Rx → Ry → Rz** (right-handed, degrees). Translation is applied **after** rotation by default (you can change it).
    
---
**Quick prerequisites**
- `py3Dmol`, `ipywidgets`, `numpy` should be installed.
- `xtb` should be in your PATH if you want energies (GFN2-xTB recommended).

In [1]:
# (Optional) one-time installs if needed (uncomment and run)
# %pip install py3Dmol ipywidgets numpy pandas
# from google.colab import output; output.enable_custom_widget_manager()  # only for Colab
    
import os, io, re, math, json, subprocess, shutil, time
import numpy as np
from pathlib import Path
from typing import List, Tuple
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import py3Dmol


## Helpers: XYZ I/O, transforms

In [2]:
def parse_xyz(text: str):
    '''Parse simple XYZ text into (symbols, coords[n,3]).'''
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if len(lines) < 3:
        raise ValueError("XYZ too short")
    try:
        n = int(lines[0])
    except Exception as e:
        raise ValueError("First line must be atom count") from e
    body = lines[2:2+n]
    if len(body) < n:
        raise ValueError("Not enough atom lines for declared count")
    symbols, xyz = [], []
    for ln in body:
        parts = ln.split()
        if len(parts) < 4:
            raise ValueError(f"Bad atom line: {ln}")
        symbols.append(parts[0])
        xyz.append([float(parts[1]), float(parts[2]), float(parts[3])])
    return symbols, np.array(xyz, dtype=float)

def to_xyz_text(symbols, coords, comment=""):
    coords = np.asarray(coords, float)
    out = [str(len(symbols)), comment]
    for s,(x,y,z) in zip(symbols, coords):
        out.append(f"{s:2s} {x: .8f} {y: .8f} {z: .8f}")
    return "\n".join(out) + "\n"

def read_xyz_from_path(path: str) -> Tuple[List[str], np.ndarray]:
    with open(path, "r", encoding="utf-8", errors="ignore") as fh:
        txt = fh.read()
    return parse_xyz(txt)

def bbox_size(coords: np.ndarray):
    mn = coords.min(axis=0)
    mx = coords.max(axis=0)
    return (mx - mn), mn, mx

def rot_x(deg: float):
    a = math.radians(deg)
    ca, sa = math.cos(a), math.sin(a)
    return np.array([[1,0,0],[0,ca,-sa],[0,sa,ca]], float)

def rot_y(deg: float):
    a = math.radians(deg)
    ca, sa = math.cos(a), math.sin(a)
    return np.array([[ca,0,sa],[0,1,0],[-sa,0,ca]], float)

def rot_z(deg: float):
    a = math.radians(deg)
    ca, sa = math.cos(a), math.sin(a)
    return np.array([[ca,-sa,0],[sa, ca,0],[0,0,1]], float)

def apply_rxyz(coords: np.ndarray, rx, ry, rz):
    R = rot_x(rx) @ rot_y(ry) @ rot_z(rz)
    return coords @ R.T

def show_axes(view, length=10.0, radius=0.15):
    '''Draw long XYZ axes as cylinders + end spheres.'''
    origin = {'x':0,'y':0,'z':0}
    # X: red
    view.addCylinder({'start':origin, 'end':{'x':length,'y':0,'z':0}, 'radius':radius, 'color':'red'})
    view.addSphere({'center':{'x':length,'y':0,'z':0}, 'radius':radius*1.8, 'color':'red'})
    # Y: green
    view.addCylinder({'start':origin, 'end':{'x':0,'y':length,'z':0}, 'radius':radius, 'color':'green'})
    view.addSphere({'center':{'x':0,'y':length,'z':0}, 'radius':radius*1.8, 'color':'green'})
    # Z: blue
    view.addCylinder({'start':origin, 'end':{'x':0,'y':0,'z':length}, 'radius':radius, 'color':'blue'})
    view.addSphere({'center':{'x':0,'y':0,'z':length}, 'radius':radius*1.8, 'color':'blue'})


## Load monomers (upload or local path)

In [110]:
# UI
u1 = widgets.FileUpload(accept='.xyz', multiple=False)
u2 = widgets.FileUpload(accept='.xyz', multiple=False)
p1 = widgets.Text(placeholder='Optional local path to monomer A .xyz')
p2 = widgets.Text(placeholder='Optional local path to monomer B .xyz (leave empty for homodimer)')
hetero = widgets.Checkbox(value=False, description='Heterodimer (use monomer B if provided)')

btn_load = widgets.Button(description='Load monomers', button_style='primary')
out_load = widgets.Output()

def _read_upload(u: widgets.FileUpload):
    if not u.value:
        return None
    # Handle both legacy dict and modern list-of-dicts
    fileinfo = None
    if isinstance(u.value, dict):
        fileinfo = list(u.value.values())[0]
    else:
        fileinfo = u.value[0]
    content = fileinfo.get('content')
    # content may be memoryview, bytes, or str
    if isinstance(content, memoryview):
        content = content.tobytes()
    if isinstance(content, (bytes, bytearray)):
        txt = content.decode('utf-8', errors='ignore')
    elif isinstance(content, str):
        txt = content
    else:
        raise TypeError(f"Unsupported uploaded content type: {type(content)}")
    return parse_xyz(txt)

def do_load(_):
    with out_load:
        clear_output()
        sym1 = xyz1 = sym2 = xyz2 = None
        try:
            res = _read_upload(u1)
            if res is None and p1.value.strip():
                res = read_xyz_from_path(p1.value.strip())
            if res is None:
                print("Provide monomer A via upload or path.")
                return
            sym1, xyz1 = res
        except Exception as e:
            print("Failed to read monomer A:", e)
            return

        if hetero.value:
            try:
                res2 = _read_upload(u2)
                if res2 is None and p2.value.strip():
                    res2 = read_xyz_from_path(p2.value.strip())
                if res2 is None:
                    print("Heterodimer selected but monomer B missing. Using A for both.")
                    sym2, xyz2 = sym1, xyz1.copy()
                else:
                    sym2, xyz2 = res2
            except Exception as e:
                print("Failed to read monomer B, using A both:", e)
                sym2, xyz2 = sym1, xyz1.copy()
        else:
            sym2, xyz2 = sym1, xyz1.copy()

        globals()['MONO_A'] = {'symbols': sym1, 'xyz': xyz1}
        globals()['MONO_B'] = {'symbols': sym2, 'xyz': xyz2}
        print(f"Loaded monomer A: {len(sym1)} atoms; monomer B: {len(sym2)} atoms")
        size, mn, mx = bbox_size(xyz1)
        L = float(max(15.0, 3.0*float(np.linalg.norm(size))))
        globals()['AXIS_LEN'] = L
        print(f"Suggested axis length: {L:.2f}")

display(widgets.VBox([
    widgets.HBox([widgets.VBox([widgets.HTML('<b>Monomer A</b>'), u1, p1]), 
                  widgets.VBox([widgets.HTML('<b>Monomer B</b>'), u2, p2])]),
    hetero, btn_load, out_load
]))
btn_load.on_click(do_load)


## Inspect monomer A orientation in py3Dmol

In [111]:
viewer = py3Dmol.view(width=700, height=500)

def show_monomer_A(style='stick', bg='white'):
    if 'MONO_A' not in globals():
        print("Load monomers first.")
        
        return
    sym, xyz = MONO_A['symbols'], MONO_A['xyz']
    L = 4.0
    viewer.removeAllModels()
    viewer.setBackgroundColor(bg)
    viewer.addModel(to_xyz_text(sym, xyz), 'xyz')
    viewer.setStyle({'stick':{}} if style=='stick' else {'sphere':{}})
    
    # Add atom number labels
    for i in range(len(sym)):
        viewer.addLabel(str(i+1),
                        {'position': {'x': float(xyz[i][0]),
                                      'y': float(xyz[i][1]),
                                      'z': float(xyz[i][2])},
                         'fontColor': 'black',
                         'backgroundColor': 'white',
                         'backgroundOpacity': 0.7,
                         'fontSize': 12})
    
    show_axes(viewer, length=L, radius=0.25)
    viewer.zoomTo()
    display(viewer.show())

widgets.interact(
    show_monomer_A,
    style=widgets.Dropdown(options=['stick','sphere'], value='stick'),
    bg=widgets.Dropdown(options=['white','black'], value='white')
);


interactive(children=(Dropdown(description='style', options=('stick', 'sphere'), value='stick'), Dropdown(desc…

In [112]:
# === OPTIONAL: Orient a monomer by two user-defined vectors
#     (A→B -> X, C→D -> Z) + ALWAYS SAVE XYZ ===
import numpy as np, os, json
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, HTML, clear_output
import py3Dmol

# Widgets
which_mono = widgets.Dropdown(options=['A','B'], value='A', description='Monomer')
a_idx = widgets.IntText(value=1, description='A (1-based)')
b_idx = widgets.IntText(value=2, description='B (1-based)')
c_idx = widgets.IntText(value=3, description='C (1-based)')
d_idx = widgets.IntText(value=4, description='D (1-based)')

center_mode = widgets.Dropdown(
    options=['centroid', 'atom A', 'ABCD centroid', 'none'],  # ← new option added
    value='centroid',
    description='Center at'
)
save_dir = widgets.Text(value='oriented_out', description='Save dir')
overwrite = widgets.Checkbox(value=True, description='Overwrite monomer coords')  # default True
btn_apply = widgets.Button(description='Apply orientation', button_style='primary')
out_orient = widgets.Output()

viewer_orient = py3Dmol.view(width=700, height=500)

def _normalize(v, eps=1e-12):
    n = np.linalg.norm(v)
    if n < eps:
        raise ValueError("Vector norm ~ 0; check your atom choices.")
    return v / n

def _make_frame(coords, iA, iB, iC, iD):
    """
    Build an orthonormal frame with:
      xhat = normalize(B - A)                # A→B defines +X
      ztmp = (D - C)                         # C→D used for Z
      zhat = normalize(ztmp - (ztmp·xhat)xhat)  # remove X component
      yhat = normalize(zhat × xhat)             # ensure right-handed frame
      (xhat, yhat, zhat) are columns of F
    """
    A = coords[iA]; B = coords[iB]; C = coords[iC]; D = coords[iD]
    xhat = _normalize(B - A)
    v2 = D - C
    v2_perp = v2 - np.dot(v2, xhat) * xhat
    zhat = _normalize(v2_perp)
    yhat = _normalize(np.cross(zhat, xhat))  # right-handed: x, y, z
    F = np.stack([xhat, yhat, zhat], axis=1)
    return F

def _apply_orientation(coords, F, center_choice, ref_point=None, iA=None, iB=None, iC=None, iD=None):
    """
    r' = (r - center) @ F   (columns of F define new axes)
    center_choice:
      - 'centroid'      : geometric centroid of all monomer atoms
      - 'atom A'        : position of atom A
      - 'ABCD centroid' : centroid of atoms A, B, C, D
      - 'none'          : no recentring (origin at 0,0,0)
    """
    if center_choice == 'centroid':
        ctr = coords.mean(axis=0)
    elif center_choice == 'atom A':
        ctr = ref_point
    elif center_choice == 'ABCD centroid':
        if None in (iA, iB, iC, iD):
            raise ValueError("ABCD centroid requires valid A, B, C, D indices.")
        ctr = (coords[iA] + coords[iB] + coords[iC] + coords[iD]) / 4.0
    else:  # 'none'
        ctr = np.zeros(3)
    return (coords - ctr) @ F

def _get_mono(which):
    if which == 'A':
        return MONO_A['symbols'], MONO_A['xyz']
    else:
        return MONO_B['symbols'], MONO_B['xyz']

def _set_mono(which, symbols, xyz_new):
    if which == 'A':
        MONO_A['symbols'] = symbols
        MONO_A['xyz'] = xyz_new
    else:
        MONO_B['symbols'] = symbols
        MONO_B['xyz'] = xyz_new

def _label_atoms(view, symbols, coords):
    for i in range(len(symbols)):
        view.addLabel(str(i+1),
                      {'position': {'x': float(coords[i,0]),
                                    'y': float(coords[i,1]),
                                    'z': float(coords[i,2])},
                       'fontColor': 'black',
                       'backgroundColor': 'white',
                       'backgroundOpacity': 0.7,
                       'fontSize': 12})

def do_orient(_):
    with out_orient:
        clear_output()
        if 'MONO_A' not in globals() or 'MONO_B' not in globals():
            print("Load monomers first.")
            return
        sym, xyz = _get_mono(which_mono.value)
        n = len(sym)
        # indices (1-based → 0-based)
        try:
            iA = a_idx.value - 1
            iB = b_idx.value - 1
            iC = c_idx.value - 1
            iD = d_idx.value - 1
            for i in (iA, iB, iC, iD):
                if i < 0 or i >= n:
                    raise IndexError("Atom index out of range.")
            if iA == iB or iC == iD:
                raise ValueError("A and B must differ; C and D must differ.")
        except Exception as e:
            print("Index error:", e)
            return

        try:
            F = _make_frame(xyz, iA, iB, iC, iD)
            ref_point = xyz[iA]
            xyz_new = _apply_orientation(
                xyz, F, center_mode.value, ref_point=ref_point,
                iA=iA, iB=iB, iC=iC, iD=iD
            )
        except Exception as e:
            print("Orientation failed:", e)
            return

        # ALWAYS save oriented XYZ
        out_dir = Path(save_dir.value).expanduser()
        out_dir.mkdir(parents=True, exist_ok=True)
        tag = 'A' if which_mono.value == 'A' else 'B'
        comment = json.dumps({
            "monomer": tag,
            "center": center_mode.value,
            "mapping": "X <- A→B ; Z <- C→D (Y rebuilt orthogonal)",
            "A": a_idx.value, "B": b_idx.value, "C": c_idx.value, "D": d_idx.value
        })
        oriented_path = out_dir / f"monomer{tag}_oriented.xyz"
        oriented_path.write_text(to_xyz_text(sym, xyz_new, comment=comment), encoding='utf-8')

        # Optionally overwrite globals (so the next grid cell uses oriented coords)
        did_overwrite = False
        if overwrite.value:
            _set_mono(which_mono.value, sym, xyz_new)
            did_overwrite = True

        # Visualize oriented monomer
        viewer_orient.removeAllModels()
        viewer_orient.setBackgroundColor('white')
        viewer_orient.addModel(to_xyz_text(sym, xyz_new, comment=f"Oriented {tag}: X←A→B, Z←C→D"), 'xyz')
        viewer_orient.setStyle({'stick':{}})
        L = 3.0
        show_axes(viewer_orient, length=L, radius=0.25)
        # _label_atoms(viewer_orient, sym, xyz_new)
        viewer_orient.zoomTo()

        display(HTML(
            f"<b>Applied to monomer {tag}</b> | Center: {center_mode.value} | "
            f"Mapping: <code>X←A→B</code>, <code>Z←C→D</code> (right-handed frame) | "
            f"Overwrite: {did_overwrite} | Saved: <code>{oriented_path}</code>"
        ))
        display(viewer_orient.show())

display(widgets.VBox([
    widgets.HTML("<b>Orient monomer so X←A→B and Z←C→D (always saves oriented XYZ; optional overwrite)</b>"),
    widgets.HBox([which_mono, center_mode, overwrite]),
    widgets.HBox([a_idx, b_idx, c_idx, d_idx]),
    widgets.HBox([save_dir]),
    btn_apply,
    out_orient
]))
btn_apply.on_click(do_orient)


## Build dimer grid

In [113]:
# === Load oriented monomers and build translational/rotational grid (with clash avoidance + PROGRESS) ===
import json, re, math
import numpy as np
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import py3Dmol

# --- Bondi-like VdW radii (Å) ---
VDW = {
    'H':1.20,'C':1.70,'N':1.55,'O':1.52,'F':1.47,
    'P':1.80,'S':1.80,'Cl':1.75,'Br':1.85,'I':1.98,
    'Si':2.10,'B':1.92,'Na':2.27,'K':2.75,'Ca':2.31,
    'Li':1.82,'Al':2.00,'Fe':2.00,'Cu':1.96,'Zn':2.01
}
def vdw_radius(sym):  # fallback
    return VDW.get(sym, 1.70)

# --- helpers (expect parse_xyz, to_xyz_text, show_axes defined) ---
def _read_xyz(path: Path):
    txt = path.read_text(encoding='utf-8', errors='ignore')
    return parse_xyz(txt)

def _parse_list(txt: str):
    if not txt.strip():
        return [0.0]
    parts = re.split(r'[\s,;]+', txt.strip())
    return [float(p) for p in parts if p]

def _rot_x(a):
    a = np.radians(a); ca, sa = np.cos(a), np.sin(a)
    return np.array([[1,0,0],[0,ca,-sa],[0,sa,ca]], float)

def _rot_y(a):
    a = np.radians(a); ca, sa = np.cos(a), np.sin(a)
    return np.array([[ca,0,sa],[0,1,0],[-sa,0,ca]], float)

def _rot_z(a):
    a = np.radians(a); ca, sa = np.cos(a), np.sin(a)
    return np.array([[ca,-sa,0],[sa, ca,0],[0,0,1]], float)

def _apply_rxyz(X, rx, ry, rz):
    R = _rot_x(rx) @ _rot_y(ry) @ _rot_z(rz)
    return X @ R.T

def _closest_pair_and_gap(Axyz, Bxyz, Asym, Bsym, safety=1.0):
    """Return (iA,iB,dmin, gap) with gap = safety*(rA+rB) - dmin (>0 means clash)."""
    dmin = 1e9; iAmin = -1; iBmin = -1; gap = 0.0
    for iA in range(Axyz.shape[0]):
        rA = vdw_radius(Asym[iA])
        Ai = Axyz[iA]
        diffs = Bxyz - Ai
        dists = np.linalg.norm(diffs, axis=1)
        j = int(np.argmin(dists))
        d = float(dists[j])
        rB = vdw_radius(Bsym[j])
        thr = safety*(rA + rB)
        g = thr - d
        if d < dmin:
            dmin, iAmin, iBmin, gap = d, iA, j, g
    return iAmin, iBmin, dmin, gap

def _separate_if_clashing(Axyz, Bxyz, Asym, Bsym, *,
                          move_B=True, safety=1.0, margin=0.05,
                          max_iter=50):
    """
    Translate moving monomer along the direction of closest A–B pair until
    all dists >= safety*(rA+rB) (+margin). Returns (Anew,Bnew,total_shift, did_separate, iters).
    """
    total_shift = np.zeros(3, float)
    Acur = Axyz.copy(); Bcur = Bxyz.copy()
    did = False
    for it in range(max_iter):
        iA, iB, d, need = _closest_pair_and_gap(Acur, Bcur, Asym, Bsym, safety=safety)
        if need <= 0:
            return Acur, Bcur, total_shift, did, it
        u = Bcur[iB] - Acur[iA]
        n = np.linalg.norm(u)
        if n < 1e-12:
            u = np.array([1.0,0.0,0.0]); n = 1.0
        u /= n
        delta = (need + margin) * u
        if move_B:
            Bcur = Bcur + delta
            total_shift += delta
        else:
            Acur = Acur - delta
            total_shift -= delta
        did = True
    return Acur, Bcur, total_shift, did, max_iter

# --- UI ---
folder = widgets.Text(value='oriented_out', description='Oriented dir')
btn_load = widgets.Button(description='Load monomers from folder', button_style='primary')
out_load = widgets.Output()

tx_txt = widgets.Text(value='0, 2.0, 4.0', description='Tx list')
ty_txt = widgets.Text(value='0', description='Ty list')
tz_txt = widgets.Text(value='0', description='Tz list')
rx_txt = widgets.Text(value='0', description='Rx° list')
ry_txt = widgets.Text(value='0', description='Ry° list')
rz_txt = widgets.Text(value='0, 15, 30', description='Rz° list')

apply_order = widgets.Dropdown(options=['Rotate then Translate','Translate then Rotate'],
                               value='Rotate then Translate', description='Order')
moved = widgets.Dropdown(options=['B (move only monomer B)','A (move only monomer A)'],
                         value='B (move only monomer B)', description='Move')
placement = widgets.Dropdown(options=['keep current positions','recenter both to centroids'],
                             value='keep current positions', description='Placement')

# clash controls
avoid_clash = widgets.Checkbox(value=True, description='Avoid clashes')
safety_factor = widgets.FloatText(value=1.0, description='Safety ×(rA+rB)')
margin_A = widgets.FloatText(value=0.05, description='Margin Å')
max_iters = widgets.IntText(value=50, description='Max iters')

# progress widgets
prog = widgets.IntProgress(value=0, min=0, max=1, description='Building:', bar_style='')
status = widgets.Label(value='')

btn_build = widgets.Button(description='Generate grid', button_style='success')
out_build = widgets.Output()

def do_load(_):
    with out_load:
        clear_output()
        d = Path(folder.value).expanduser()
        a_path = d / 'monomerA_oriented.xyz'
        b_path = d / 'monomerB_oriented.xyz'
        if not a_path.exists():
            print(f"Missing {a_path}")
            return
        symA, xyzA = _read_xyz(a_path)
        if b_path.exists():
            symB, xyzB = _read_xyz(b_path)
            print("Loaded heterodimer from oriented files.")
        else:
            symB, xyzB = symA, xyzA.copy()
            print("Loaded homodimer (B not found; using A for both).")
        globals()['MONO_A'] = {'symbols': symA, 'xyz': xyzA}
        globals()['MONO_B'] = {'symbols': symB, 'xyz': xyzB}
        v = py3Dmol.view(width=600, height=420)
        v.setBackgroundColor('white')
        v.addModel(to_xyz_text(symA, xyzA, comment='monomer A'), 'xyz')
        v.setStyle({'stick':{}})
        L = float(max(6.0, 2.0*np.linalg.norm((xyzA.max(axis=0)-xyzA.min(axis=0)))))
        show_axes(v, length=L, radius=0.25)
        v.zoomTo()
        display(HTML("<b>Monomer A preview</b>"))
        display(v.show())

def make_grid(_):
    with out_build:
        clear_output()

        if 'MONO_A' not in globals() or 'MONO_B' not in globals():
            print("Load monomers first (use the button above).")
            return

        A = {'symbols': MONO_A['symbols'], 'xyz': MONO_A['xyz'].copy()}
        B = {'symbols': MONO_B['symbols'], 'xyz': MONO_B['xyz'].copy()}

        # placement
        if placement.value.startswith('recenter'):
            Axyz0 = A['xyz'] - A['xyz'].mean(axis=0)
            Bxyz0 = B['xyz'] - B['xyz'].mean(axis=0)
        else:
            Axyz0 = A['xyz']; Bxyz0 = B['xyz']

        txs = _parse_list(tx_txt.value)
        tys = _parse_list(ty_txt.value)
        tzs = _parse_list(tz_txt.value)
        rxs = _parse_list(rx_txt.value)
        rys = _parse_list(ry_txt.value)
        rzs = _parse_list(rz_txt.value)

        total = len(txs)*len(tys)*len(tzs)*len(rxs)*len(rys)*len(rzs)
        prog.max = max(1, total)
        prog.value = 0
        prog.bar_style = ''  # normal
        status.value = f"0 / {total}"

        display(widgets.VBox([prog, status]))

        dimers, meta = [], []
        move_B = moved.value.startswith('B')
        rot_then_trans = apply_order.value.startswith('Rotate')

        done = 0
        for tx in txs:
            for ty in tys:
                for tz in tzs:
                    for rx in rxs:
                        for ry in rys:
                            for rz in rzs:
                                # fresh copies
                                Axyz = Axyz0.copy()
                                Bxyz = Bxyz0.copy()
                                if move_B:
                                    if rot_then_trans:
                                        Bxyz = _apply_rxyz(Bxyz, rx, ry, rz)
                                        Bxyz = Bxyz + np.array([tx,ty,tz])
                                    else:
                                        Bxyz = Bxyz + np.array([tx,ty,tz])
                                        Bxyz = _apply_rxyz(Bxyz, rx, ry, rz)
                                else:
                                    if rot_then_trans:
                                        Axyz = _apply_rxyz(Axyz, rx, ry, rz)
                                        Axyz = Axyz + np.array([tx,ty,tz])
                                    else:
                                        Axyz = Axyz + np.array([tx,ty,tz])
                                        Axyz = _apply_rxyz(Axyz, rx, ry, rz)

                                extra_shift = np.zeros(3); resolved = False; iters = 0
                                if avoid_clash.value:
                                    Axyz, Bxyz, shift_vec, did, iters = _separate_if_clashing(
                                        Axyz, Bxyz, A['symbols'], B['symbols'],
                                        move_B=move_B, safety=safety_factor.value,
                                        margin=margin_A.value, max_iter=max_iters.value
                                    )
                                    extra_shift = shift_vec; resolved = did

                                sym = A['symbols'] + B['symbols']
                                coords = np.vstack([Axyz, Bxyz])
                                dimers.append((sym, coords))
                                meta.append({
                                    'tx':tx,'ty':ty,'tz':tz,'rx':rx,'ry':ry,'rz':rz,
                                    'moved':'B' if move_B else 'A',
                                    'order':'RthenT' if rot_then_trans else 'TthenR',
                                    'placement': placement.value,
                                    'avoid_clash': bool(avoid_clash.value),
                                    'safety': safety_factor.value,
                                    'margin_A': margin_A.value,
                                    'iters': iters,
                                    'resolved': bool(resolved),
                                    'extra_shift': extra_shift.tolist()
                                })

                                done += 1
                                if done % 10 == 0 or done == total:
                                    prog.value = done
                                    status.value = f"{done} / {total}"

        globals()['DIMER_SET'] = dimers
        globals()['DIMER_META'] = meta
        prog.value = total
        status.value = f"Done: {total} / {total}"
        prog.bar_style = 'success'

        print(f"Generated {len(dimers)} dimers from folder: {folder.value}")
        if dimers:
            L = 3.0
            v = py3Dmol.view(width=600, height=420)
            v.setBackgroundColor('white')
            v.addModel(to_xyz_text(*dimers[0], comment=json.dumps(meta[0])), 'xyz')
            v.setStyle({'stick':{}})
            show_axes(v, length=L, radius=0.25)
            v.zoomTo()
            display(HTML("<b>Preview of dimer #0</b>"))
            display(v.show())

display(widgets.VBox([
    widgets.HTML("<b>Load oriented monomers and build dimer grid (with clash avoidance & progress)</b>"),
    widgets.HBox([folder, btn_load]),
    out_load,
    widgets.HBox([tx_txt, ty_txt, tz_txt]),
    widgets.HBox([rx_txt, ry_txt, rz_txt]),
    widgets.HBox([apply_order, moved, placement]),
    widgets.HBox([avoid_clash, safety_factor, margin_A, max_iters]),
    btn_build,
    out_build
]))
btn_load.on_click(do_load)
btn_build.on_click(make_grid)


## Preview / export dimers

In [109]:
# === Export grid dimers to XYZ files ===
import os, json, gzip
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output

# UI controls
out_dir = widgets.Text(value='dimers_out', description='Output dir')
fname_pattern = widgets.Text(
    value='idx{idx:04d}_tx{tx}_ty{ty}_tz{tz}_rx{rx}_ry{ry}_rz{rz}_{moved}_{order}.xyz',
    description='Name pattern',
    layout=widgets.Layout(width='95%')
)
include_meta = widgets.Checkbox(value=True, description='Include metadata in XYZ comment line')
use_gzip = widgets.Checkbox(value=False, description='Gzip output (.xyz.gz)')
overwrite = widgets.Checkbox(value=True, description='Overwrite if exists')
sel_idx = widgets.IntText(value=0, description='Selected idx')

btn_export_all = widgets.Button(description='Export ALL', button_style='primary')
btn_export_sel = widgets.Button(description='Export selected', button_style='info')

prog = widgets.IntProgress(value=0, min=0, max=1, description='Writing:')
status = widgets.Label(value='')
out_log = widgets.Output()

def _make_name(meta, idx):
    name = fname_pattern.value.format(idx=idx, **meta)
    if use_gzip.value and not name.endswith('.gz'):
        if name.endswith('.xyz'):
            name = name + '.gz'
        else:
            name = name + '.xyz.gz'
    return name

def _write_xyz(path: Path, symbols, coords, meta_txt: str):
    text = to_xyz_text(symbols, coords, comment=meta_txt)
    if use_gzip.value:
        with gzip.open(path, 'wt', encoding='utf-8') as f:
            f.write(text)
    else:
        path.write_text(text, encoding='utf-8')

def _export_indices(indices):
    with out_log:
        clear_output()
        if 'DIMER_SET' not in globals() or not DIMER_SET:
            print("No dimers to export. Generate the grid first.")
            return
        d = Path(out_dir.value).expanduser()
        d.mkdir(parents=True, exist_ok=True)

        prog.max = len(indices)
        prog.value = 0
        status.value = f"0 / {len(indices)}"
        display(widgets.VBox([prog, status]))

        written = 0
        skipped = 0
        for k,i in enumerate(indices, start=1):
            sym, coords = DIMER_SET[i]
            meta = DIMER_META[i]
            meta_txt = json.dumps(meta) if include_meta.value else ''
            name = _make_name(meta, i)
            path = d / name
            if path.exists() and not overwrite.value:
                skipped += 1
            else:
                _write_xyz(path, sym, coords, meta_txt)
                written += 1
            prog.value = k
            status.value = f"{k} / {len(indices)}"

        print(f"Done. Written: {written}, Skipped (exists): {skipped}")
        print(f"Output folder: {d.resolve()}")

def do_export_all(_):
    if 'DIMER_SET' not in globals() or not DIMER_SET:
        with out_log:
            clear_output()
            print("No dimers to export. Generate the grid first.")
            return
    _export_indices(list(range(len(DIMER_SET))))

def do_export_sel(_):
    if 'DIMER_SET' not in globals() or not DIMER_SET:
        with out_log:
            clear_output()
            print("No dimers to export. Generate the grid first.")
            return
    i = sel_idx.value
    if i < 0 or i >= len(DIMER_SET):
        with out_log:
            clear_output()
            print(f"Selected idx {i} is out of range (0..{len(DIMER_SET)-1}).")
            return
    _export_indices([i])

btn_export_all.on_click(do_export_all)
btn_export_sel.on_click(do_export_sel)

display(widgets.VBox([
    widgets.HTML("<b>Export dimers to XYZ</b>"),
    out_dir,
    fname_pattern,
    widgets.HBox([include_meta, use_gzip, overwrite]),
    widgets.HBox([btn_export_all, sel_idx, btn_export_sel]),
    out_log
]))


In [56]:
# === Preview exported dimers (licorice style) from a folder ===
import os, json, gzip, re
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import py3Dmol
import numpy as np

# --- helpers (expects to_xyz_text, show_axes already defined, but not required here) ---
def _read_text_any(path: Path) -> str:
    if path.suffix == '.gz':
        with gzip.open(path, 'rt', encoding='utf-8', errors='ignore') as f:
            return f.read()
    return path.read_text(encoding='utf-8', errors='ignore')

def _xyz_to_py3dmol(text: str):
    """Return (symbols, coords ndarray, meta_obj_or_text)."""
    lines = [l.rstrip('\n') for l in text.splitlines()]
    if len(lines) < 2:
        raise ValueError("XYZ file too short.")
    try:
        n = int(lines[0].strip())
    except:
        raise ValueError("First line must be atom count.")
    comment = lines[1] if len(lines) > 1 else ""
    body = lines[2:2+n]
    symbols, coords = [], []
    for ln in body:
        parts = ln.split()
        if len(parts) < 4:
            continue
        symbols.append(parts[0])
        coords.append([float(parts[1]), float(parts[2]), float(parts[3])])
    coords = np.array(coords, float) if coords else np.zeros((0,3))
    # parse comment as JSON if possible
    meta = comment
    try:
        meta_obj = json.loads(comment)
        meta = meta_obj
    except Exception:
        pass
    return symbols, coords, meta

def _update_file_list():
    d = Path(folder.value).expanduser()
    patt = re.compile(filter_pat.value) if filter_pat.value.strip() else None
    files = []
    if d.exists() and d.is_dir():
        for p in sorted(d.iterdir()):
            if p.is_file() and (p.suffix.lower() in ('.xyz',) or (p.suffix.lower()=='.gz' and p.name.endswith('.xyz.gz'))):
                if patt is None or patt.search(p.name):
                    files.append(p)
    file_list.options = [str(p) for p in files]
    if files:
        file_list.index = 0

def _show_selected(path_str: str):
    with out_view:
        clear_output()
        if not path_str:
            print("No file selected.")
            return
        p = Path(path_str)
        try:
            text = _read_text_any(p)
            sym, xyz, meta = _xyz_to_py3dmol(text)
        except Exception as e:
            print("Failed to read:", p, "\n", e)
            return
        v = py3Dmol.view(width=viewer_width.value, height=viewer_height.value)
        v.setBackgroundColor(bg.value)
        v.addModel(text, 'xyz')
        # licorice-like sticks
        v.setStyle({'stick': {'radius': float(stick_radius.value)}})
        # optional axes
        if show_axes_chk.value:
            # auto axis length by bbox
            if xyz.size:
                L = 3.0
            else:
                L = 3.0
            # draw axes using the same helper if available; otherwise inline
            try:
                show_axes(v, length=L, radius=0.25)
            except NameError:
                origin = {'x':0,'y':0,'z':0}
                v.addCylinder({'start':origin, 'end':{'x':L,'y':0,'z':0}, 'radius':0.25, 'color':'red'})
                v.addCylinder({'start':origin, 'end':{'x':0,'y':L,'z':0}, 'radius':0.25, 'color':'green'})
                v.addCylinder({'start':origin, 'end':{'x':0,'y':0,'z':L}, 'radius':0.25, 'color':'blue'})
        v.zoomTo()
        if isinstance(meta, dict):
            meta_html = "<pre style='white-space:pre-wrap;margin:0'>" + json.dumps(meta, indent=2) + "</pre>"
        else:
            meta_html = f"<code style='white-space:pre-wrap'>{meta}</code>"
        display(HTML(f"<b>File:</b> {p.name}"))
        display(HTML(f"<b>Metadata:</b> {meta_html}"))
        display(v.show())

def _on_select(change):
    if change['name'] == 'value' and change['new']:
        _show_selected(change['new'])

def _prev(_):
    if not file_list.options:
        return
    i = file_list.index
    file_list.index = (i - 1) % len(file_list.options)

def _next(_):
    if not file_list.options:
        return
    i = file_list.index
    file_list.index = (i + 1) % len(file_list.options)

def _refresh(_):
    _update_file_list()
    if file_list.options:
        _show_selected(file_list.value)

# --- UI widgets ---
folder = widgets.Text(value='dimers_out', description='Folder')
filter_pat = widgets.Text(value='', description='Filter (regex)', layout=widgets.Layout(width='60%'))
refresh_btn = widgets.Button(description='Refresh list', button_style='warning')

file_list = widgets.Dropdown(options=[], description='Dimer file', layout=widgets.Layout(width='95%'))
prev_btn = widgets.Button(description='⟨ Prev')
next_btn = widgets.Button(description='Next ⟩')

bg = widgets.Dropdown(options=['white','black'], value='white', description='Background')
stick_radius = widgets.FloatText(value=0.2, description='Stick radius')
show_axes_chk = widgets.Checkbox(value=True, description='Show axes')
viewer_width = widgets.IntText(value=700, description='Width')
viewer_height = widgets.IntText(value=500, description='Height')

out_view = widgets.Output()

# wiring
file_list.observe(_on_select, names='value')
prev_btn.on_click(_prev)
next_btn.on_click(_next)
refresh_btn.on_click(_refresh)

# initial build & display UI
_update_file_list()
controls_top = widgets.HBox([folder, filter_pat, refresh_btn])
controls_mid = widgets.HBox([file_list])
controls_vis = widgets.HBox([bg, stick_radius, show_axes_chk, viewer_width, viewer_height])
nav = widgets.HBox([prev_btn, next_btn])

display(widgets.VBox([
    widgets.HTML("<b>Preview dimers from folder (licorice style)</b>"),
    controls_top,
    controls_mid,
    controls_vis,
    nav,
    out_view
]))

# show the first file if present
if file_list.options:
    _show_selected(file_list.value)
